In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras import backend as K

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;


modelPath = '../models/VGG19_opt/run_sgd.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="1"

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)
a = keras.optimizers.Adam(lr=.4);
print(a.lr)

<tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
#     with tf.device('/device:GPU:0'):
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    K.set_session(sess)

    base_model = VGG19(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)):
        base_model.layers[i].trainable = False

    # optimization
    lr={{uniform(0.00001,0.01)}};
    momentum={{uniform(0,0.01)}}
    decay={{uniform(0,0.001)}};
    nesterov={{choice([True,False])}}

    print()
    print('lr=',lr);
    print('momentum=',momentum);
    print('decay=',decay);
    print('nesterov=',nesterov);
    print()

    sgd = keras.optimizers.sgd(lr=lr,momentum=momentum,nesterov=nesterov,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=sgd, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));

    print('Tensorboard NOT activated')

    model.fit(X_train, Y_train,
      epochs=100,
      batch_size = 64,
      validation_data=(X_valid, Y_valid),
      verbose=1,
      callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='VGG19Optimization_sgd')

X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg19 import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
    pass

try:
    from subpr

100%|██████████| 10222/10222 [00:49<00:00, 207.08it/s]


Splitting into training/validation
Creating model

lr= 0.006928464952814032
momentum= 0.0021280043312755827
decay= 0.0007342146978592598
nesterov= True

Tensorboard NOT activated
Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 38s 5ms/step - loss: 4.8808 - acc: 0.0098 - val_loss: 4.7854 - val_acc: 0.0124
Epoch 2/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.8284 - acc: 0.0103 - val_loss: 4.7701 - val_acc: 0.0166
Epoch 3/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.7949 - acc: 0.0133 - val_loss: 4.7617 - val_acc: 0.0209
Epoch 4/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.7793 - acc: 0.0162 - val_loss: 4.7548 - val_acc: 0.0225
Epoch 5/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.7677 - acc: 0.0164 - val_loss: 4.7454 - val_acc: 0.0277
Epoch 6/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.7570 - acc: 0

7155/7155 [==============================] - 25s 4ms/step - loss: 4.2258 - acc: 0.0903 - val_loss: 4.3131 - val_acc: 0.0841
Epoch 60/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2122 - acc: 0.0868 - val_loss: 4.3077 - val_acc: 0.0818
Epoch 61/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2099 - acc: 0.0914 - val_loss: 4.3043 - val_acc: 0.0825
Epoch 62/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2012 - acc: 0.0897 - val_loss: 4.3011 - val_acc: 0.0828
Epoch 63/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.1892 - acc: 0.0955 - val_loss: 4.2960 - val_acc: 0.0848
Epoch 64/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.1876 - acc: 0.0963 - val_loss: 4.2917 - val_acc: 0.0864
Epoch 65/100
7155/7155 [==============================] - 26s 4ms/step - loss: 4.1938 - acc: 0.0872 - val_loss: 4.2847 - val_acc: 0.0841
Epoch 66/100
7155/7155 [==============================

7155/7155 [==============================] - 25s 4ms/step - loss: 4.6064 - acc: 0.0422 - val_loss: 4.6161 - val_acc: 0.0489
Epoch 32/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.5933 - acc: 0.0433 - val_loss: 4.6111 - val_acc: 0.0476
Epoch 33/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.5882 - acc: 0.0456 - val_loss: 4.6047 - val_acc: 0.0473
Epoch 34/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.5846 - acc: 0.0454 - val_loss: 4.6002 - val_acc: 0.0492
Epoch 35/100
7155/7155 [==============================] - 26s 4ms/step - loss: 4.5749 - acc: 0.0485 - val_loss: 4.5963 - val_acc: 0.0483
Epoch 36/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.5767 - acc: 0.0458 - val_loss: 4.5895 - val_acc: 0.0522
Epoch 37/100
7155/7155 [==============================] - 26s 4ms/step - loss: 4.5656 - acc: 0.0458 - val_loss: 4.5828 - val_acc: 0.0528
Epoch 38/100
7155/7155 [==============================

...

7155/7155 [==============================] - 25s 4ms/step - loss: 4.6993 - acc: 0.0259 - val_loss: 4.6970 - val_acc: 0.0300
Epoch 26/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6864 - acc: 0.0300 - val_loss: 4.6927 - val_acc: 0.0284
Epoch 27/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6798 - acc: 0.0317 - val_loss: 4.6854 - val_acc: 0.0297
Epoch 28/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6802 - acc: 0.0302 - val_loss: 4.6800 - val_acc: 0.0316
Epoch 29/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6772 - acc: 0.0298 - val_loss: 4.6762 - val_acc: 0.0333
Epoch 30/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6695 - acc: 0.0330 - val_loss: 4.6701 - val_acc: 0.0346
Epoch 31/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.6660 - acc: 0.0338 - val_loss: 4.6653 - val_acc: 0.0336
Epoch 32/100
7155/7155 [==============================

Epoch 85/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2584 - acc: 0.0787 - val_loss: 4.3268 - val_acc: 0.0786
Epoch 86/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2411 - acc: 0.0886 - val_loss: 4.3226 - val_acc: 0.0769
Epoch 87/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2306 - acc: 0.0885 - val_loss: 4.3216 - val_acc: 0.0766
Epoch 88/100
7155/7155 [==============================] - 26s 4ms/step - loss: 4.2268 - acc: 0.0861 - val_loss: 4.3136 - val_acc: 0.0825
Epoch 89/100
7155/7155 [==============================] - 26s 4ms/step - loss: 4.2262 - acc: 0.0860 - val_loss: 4.3103 - val_acc: 0.0796
Epoch 90/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2179 - acc: 0.0936 - val_loss: 4.3028 - val_acc: 0.0799
Epoch 91/100
7155/7155 [==============================] - 25s 4ms/step - loss: 4.2195 - acc: 0.0887 - val_loss: 4.2968 - val_acc: 0.0792
Epoch 92/100
7155/7155 [=================

100%|██████████| 10222/10222 [00:49<00:00, 206.04it/s]


Splitting into training/validation


MemoryError: 

In [ ]:
best_model.save(modelPath);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)